In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dmt.tk.plotting.shapes import Circle, Arc, PolarPoint
from dmt.tk.plotting.circle_plot import NetworkGeom, NodeGeom

In [2]:
gcircle = NetworkGeom(label = "Flow-Chart")

In [3]:
figure = plt.figure(figsize=(12, 12))
lines = gcircle.draw()
gcircle.arc(0., np.pi / 4.).draw()

TypeError: draw() missing 1 required positional argument: 'data'

<Figure size 864x864 with 0 Axes>

In [ ]:
from matplotlib.collections import PolyCollection, PatchCollection
from matplotlib.patches import Polygon as PatchPolygon
from dmt.tk.plotting.shapes import Polygon
gcircle = NetworkGeom(label="Circle of Flows")
figure = plt.figure(figsize=(12, 12))
lines = gcircle.draw()
ax = plt.gca()
node_start =\
    NodeGeom(
        label="start",
        position=PolarPoint(gcircle.radius, 0.),
        thickness=PolarPoint(0.1, np.pi / 4.),
        network_geom=gcircle)
node_end =\
    NodeGeom(
        label="start",
        position=PolarPoint(gcircle.radius, -np.pi/2.),
        thickness=PolarPoint(0.1, np.pi / 4.),
        network_geom=gcircle)
node_start.draw()
node_end.draw()

Below we will digest *MR*'s code.

In [ ]:
mtypes = ["L23_MC", "L5_TPC:A", "L6_UPC"]
pathways = [
    (pre_mtype, post_mtype)
    for pre_mtype in mtypes
    for post_mtype in mtypes]
N = len(pathways)
pathway_strength = pd.DataFrame(
    {("pathway_strength", "mean"): np.random.uniform(size=N),
     ("pathway_strength", "std"):  np.random.uniform(size=N)},
    index=pd.MultiIndex.from_tuples(
        pathways,
        names=["pre_mtype", "post_mtype"]))
def get_node_weights(dataframe):
    weights = dataframe.groupby("pre_mtype").agg("sum")[("pathway_strength", "mean")]
    weights.index.name = "label"
    weights.name = "weight"
    return weights/np.sum(weights)
node_weights(pathway_strength)

In [ ]:
from dmt.tk.plotting.circle_plot import FlowGeom
node_weights = get_node_weights(pathway_strength)
ps = pathway_strength.reset_index()
link_weights =\
    pd.DataFrame(
        {"weight": ps.apply(
                       lambda row: row["pathway_strength"]["mean"],
                       axis=1
                   ).values/node_weights[ps["pre_mtype"]].values},
        index=pathway_strength.index)
for x, y in link_weights.iterrows():
    print(y)

In [ ]:
from collections import OrderedDict
network_geom.nodes = OrderedDict([
    (node.label, node)
    for node in network_geom.children])
network_geom.nodes
network_geom.flows = [
    FlowGeom(
        label="flow",
        angular_thickness=weight,
        begin_node_geom=network_geom.nodes[begin_node_label],
        end_node_geom=network_geom.nodes[end_node_label],
        network_geom=network_geom)
    for (begin_node_label, end_node_label), weight in link_weights.items()]

In [ ]:
node_weights

In [ ]:
network_geom = NetworkGeom(label="Example")
network_geom.spawn_nodes(get_node_weights(pathway_strength))

In [ ]:
figure = plt.figure(figsize=(12, 12))
network_geom.draw()
for node_geom in network_geom.node_geoms.values():
    print(node_geom.position.angular)
for flow in network_geom.flow_geoms:
    flow.draw()

In [ ]:
import numpy as np
import pandas as pd
from dmt.tk.field import lazyfield
from neuro_dmt import terminology

class BlueBrainCircuitModel:
    """
    Wrap bluepy Circuit, to provide additional methods.
    """
    def __init__(self, bluepy_circuit):
        """
        ...
        """
        self._bluepy_circuit = bluepy_circuit
        
    def __getattr__(self, attr):
        """
        ...
        """
        return getattr(self._bluepy_circuit, attr)
    
    @lazyfield
    def mtypes(self):
        """
        A list of all the mtypes.
        """
        return self.mtype_gid.unique().astype(str)
        
    @lazyfield
    def mtype_gid(self):
        """
        Mtype of a given GID.
        """
        return self.cells.get(properties=terminology.cell.mtype)
                
        
    def post_synaptic_mtype_counts(self, gid):
        """..."""
        counts_mtype_post =\
            self.mtype_gid[
                self.connectome.efferent_synapses(
                    gid,
                    properties=terminology.synapse.post_gid)
            ].value_counts()
        counts_mtype_post.index.name = "post_mtype"
        counts_mtype_post.name = "synapse_count"
        return pd\
            .DataFrame(counts_mtype_post)\
            .reset_index()\
            .assign(pre_gid=gid)\
            .set_index(["pre_gid", "post_mtype"])
    
    def pre_synaptic_mtype_counts(self, gid):
        """..."""
        counts_mtype_pre =\
            self.mtype_gid[
                self.connectome.afferent_synapses(
                    gid,
                    properties=terminology.synapse.pre_gid)
            ].value_counts()
        counts_mtype_pre.index.name = "pre_mtype"
        counts_mtype_pre.name = "synapse_count"
        return pd\
            .DataFrame(counts_mtype_pre)\
            .reset_index()\
            .assign(post_gid=gid)\
            .set_index(["pre_mtype", "post_gid"])

In [ ]:
from bluepy.v2.enums import Cell, Synapse
from bluepy.v2.circuit import Circuit
from neuro_dmt import terminology

class BlueBrainModelAdapter:
    
    def get_mtypes(self, circuit):
        """
        All mtypes in the given circuit.
        """
        return circuit.cells.get(properties=terminology.cell.mtype).unique()

    def get_synapse_count(self, circuit, direction):
        """
        
        Arguments
        -----------------
        direction :: 
        """
        mtypes = self.get_mtypes(circuit)
        if direction == terminology.direction.forward:
            pre_mtypes = mtypes
            post_mtypes = mtypes
            
        pass

In [ ]:
#circuit = Circuit(
 #   os.path.join(
 #       "/gpfs/bbp.cscs.ch/project/proj64/dissemination/circuits/S1/juvenile",
  #      "L23_MC_BTC_shifted_down/Bio_1/20190903/CircuitConfig"))
#bbcm = BlueBrainCircuitModel(circuit)

In [ ]:
#post_mtype_counts_1 = bbcm.post_synaptic_mtype_counts(1)

In [ ]:
#post_mtype_counts_1.head()

We have some idea of what data this analysis requires --- we will code that in due time.
Meanwhile, we will now focus on the drawing aspect.

In [ ]:
from collections import namedtuple
#Link = namedtuple("Link", )
Point = namedtuple('Point', ['x', 'y'], verbose=True)

In [ ]:
class Synaptome:
    
    
    
    @lazyfield
    def link_starts(self):
        """
        Example
        """
        return {
            "AFF:L23_MC": (np.array(0.0, 0.0), np.array(0.0, 0.5)),
            "AFF:L23_MC": (np.array())}
        
    @lazyfield
    def link_collection(self):
        """
        Example
        """
        return 

In [ ]:
from collections import OrderedDict
from matplotlib import pyplot as plt
from matplotlib.collections import PolyCollection
ax = plt.gca()
something = {"x": (0., 0.5), "y": (0.5, 1.0)}
vertices = OrderedDict([ 
    (0, (0., 0.5)),
    (1, (0.5, 1.0)),
    (2, (1.0, 0.5)),
    (3, (0.5, 0.))])
edges = [
    (vertices[0], vertices[1]),
    (vertices[1], vertices[2]),
    (vertices[2], vertices[3]),
    (vertices[3], vertices[0])]
link_collection =\
    PolyCollection(
        edges,
        closed=True,
        facecolors=["red", "green"],
        edgecolors="grey",
        linewidth=1.)
ax.add_collection(
    link_collection)

for name, position in vertices.items():
    ax.text(position[0], position[1], name)
plt.axis("off")